In [1]:
import optuna
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

c:\Users\27904\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization
importlib.reload(data)

<module 'data' from 'c:\\Users\\27904\\OneDrive\\桌面\\crypto\\AAA_Thursday1\\data\\__init__.py'>

In [3]:
df = data.load_asset('BTCUSDT', sampling='1d')
df = data.add_fear_and_greed(df)
# Restrict data after 2022
df = df[df.index >= '2020-01-01']
df['Return'] = np.log(df['Close'] / df['Close'].shift(1))
df.head()

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,F&G,F&G category,Return
Open time,,,,,,,,,,,,
2020-01-01,7195.24,7255.0,7175.15,7200.85,16792.388165,1.212145e+08,194010,8946.955535,6.459779e+07,37.0,Fear,NaN
2020-01-02,7200.77,7212.5,6924.74,6965.71,31951.483932,2.259823e+08,302667,15141.611340,1.070608e+08,39.0,Fear,-0.033200
2020-01-03,6965.49,7405.0,6871.04,7344.96,68428.500451,4.950986e+08,519854,35595.496273,2.577131e+08,38.0,Fear,0.053015
2020-01-04,7345.00,7404.0,7272.21,7354.11,29987.974977,2.198742e+08,279370,16369.382248,1.200351e+08,38.0,Fear,0.001245
2020-01-05,7354.19,7495.0,7318.00,7358.75,38331.085604,2.848487e+08,329209,19455.369564,1.446001e+08,39.0,Fear,0.000631


In [5]:
import numpy as np
import pandas as pd
from arch import arch_model
from sklearn.metrics import mean_squared_error

# Assume your DataFrame looks like this
# df['Return']: the return series (daily, hourly, etc.)

# Optional: Fill NaNs if needed
df['Return'] = df['Return'].fillna(0)

# === Step 1: Split into train and test ===
train_size = int(len(df) * 0.8)
train_returns = df['Return'].iloc[:train_size]
test_returns = df['Return'].iloc[train_size:]

# === Step 2: Fit ARMA-GARCH model on train ===
# ARMA(p, q) + GARCH(1,1)
model = arch_model(train_returns, vol='GARCH', p=1, q=1, mean='ARX', lags=1, dist='normal')
model_fit = model.fit(disp='off')

# === Step 3: Forecast the next len(test) steps ===
# dynamic=True uses past forecasted values rather than actuals
forecast = model_fit.forecast(horizon=len(test_returns), start=train_size, reindex=False)

# === Step 4: Extract mean predictions ===
# Since we're forecasting the conditional mean of returns
predicted_mean = forecast.mean.iloc[-1].values

# === Step 5: Compute MSE ===
# Ensure predicted_mean and test_returns are same length
mse = mean_squared_error(test_returns.values, predicted_mean)

print(f"ARMA-GARCH MSE: {mse:.6f}")


c:\Users\27904\AppData\Local\Programs\Python\Python312\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001284. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


IndexError: single positional indexer is out-of-bounds

In [5]:
import numpy as np
import pandas as pd
import ta  # pip install ta

def compute_slope(series):
    y = series.values
    x = np.arange(len(y))
    if len(y) == 0:
        return np.nan
    return np.polyfit(x, y, 1)[0]

def add_technical_features(df, look_back=14):
    df = df.copy()  # avoid modifying original

    # Return: Daily log return
    df['Return'] = np.log(df['Close'] / df['Close'].shift(1))

    # Risk: Rolling std dev (volatility)
    df['Risk'] = df['Return'].rolling(window=look_back).std()

    # RSI
    df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=look_back).rsi()

    # ADX
    df['ADX'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=look_back).adx()

    # Momentum
    df['MOM'] = df['Close'] - df['Close'].shift(look_back)

    # High-low spread
    df['HL'] = df['High'] - df['Low']

    # High-open & low-open
    df['HO'] = df['High'] - df['Open']
    df['LO'] = df['Low'] - df['Open']

    # Buy pressure ratio
    df['buy_pressure_ratio'] = df['Taker buy quote asset volume'] / df['Quote asset volume']

    # Trades per volume
    df['trades_per_volume'] = df['Number of trades'] / df['Quote asset volume']

    # Rolling slope
    df['slope'] = df['Close'].rolling(window=look_back).apply(compute_slope, raw=False)

    # Drop NaN from lookback window
    df = df.dropna()

    # Drop unnecessary columns
    cols_to_drop = [
        'Open', 'High', 'Low', 'Quote asset volume',
        'Number of trades', 'Taker buy base asset volume',
        'Taker buy quote asset volume', 'F&G'
    ]
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

    return df


In [6]:
df = add_technical_features(df)
df.head()

,Close,Volume,F&G category,Return,Risk,RSI,ADX,MOM,HL,HO,LO,buy_pressure_ratio,trades_per_volume,slope
Open time,,,,,,,,,,,,,,
2020-01-15,8821.41,84816.297606,Neutral,0.001293,0.036403,76.896910,0.0,1620.56,352.48,101.84,-250.64,0.501230,0.001027,118.805231
2020-01-16,8720.01,51991.074284,Greed,-0.011561,0.034638,73.142915,0.0,1754.30,273.81,39.80,-234.01,0.500086,0.001202,109.919824
2020-01-17,8913.28,70897.737377,Neutral,0.021922,0.033046,75.589051,0.0,1568.32,369.21,321.50,-47.71,0.509868,0.001074,111.347956
2020-01-18,8915.96,38294.746545,Neutral,0.000301,0.033074,75.622208,0.0,1561.85,182.50,75.61,-106.89,0.507244,0.001344,106.260593
2020-01-19,8701.70,70676.889259,Neutral,-0.024325,0.034479,67.704574,0.0,1342.95,732.98,283.89,-449.09,0.476452,0.000987,88.339209


In [7]:
def prep_data(df, sequence_length=30, test_size=0.2):
    # 1) include 'Close' since that's what we'll predict
    features = [
        'Close',      # ← our prediction target
        'Volume', 'F&G category', 'Return', 'Risk', 'RSI', 'ADX',
        'MOM', 'HL', 'HO', 'LO',
        'buy_pressure_ratio', 'trades_per_volume', 'slope'
    ]
    
    # 2) split numeric vs cat\egorical
    num_feats = [f for f in features if f != 'F&G category']
    df_num = df[num_feats]
    df_cat = pd.get_dummies(df['F&G category'], prefix='F_G')
    
    # 3) combine & scale
    df_all = pd.concat([df_num, df_cat], axis=1).dropna()
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df_all.values)
    feature_names = df_all.columns.tolist()
    
    # 4) build sequences for time-series
    target_idx = feature_names.index('Close')
    X, y = [], []
    for i in range(sequence_length, len(data_scaled)):
        X.append(data_scaled[i-sequence_length:i])
        # y is the scaled Close price at time i (next day)
        y.append(data_scaled[i, target_idx])
    X, y = np.array(X), np.array(y)
    
    # 5) train/test split (preserving time order)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, shuffle=False
    )
    return (X_train, X_test, y_train, y_test), scaler

In [12]:
# 2. Build LSTM model given an Optuna trial
def create_model(trial, input_shape):
    model = Sequential()
    # choose #layers
    n_layers = trial.suggest_int('n_layers', 1, 3)
    for i in range(n_layers):
        # units per layer
        units = trial.suggest_int(f'n_units_l{i}', 32, 256, step=32)
        # return sequences except for last LSTM
        ret_seq = (i < n_layers - 1)
        if i == 0:
            model.add(LSTM(units, return_sequences=ret_seq, input_shape=input_shape))
        else:
            model.add(LSTM(units, return_sequences=ret_seq))
        # dropout
        d = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5, step=0.1)
        if d > 0:
            model.add(Dropout(d))
    model.add(Dense(1, activation='linear'))
    
    # optimizer & LR
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    opt_name = trial.suggest_categorical('optimizer', ['Adam','RMSprop'])
    optimizer = Adam(lr) if opt_name=='Adam' else RMSprop(lr)
    
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# 3. Optuna objective
def objective(trial):
    (X_train, X_val, y_train, y_val), _ = prep_data(df, sequence_length=SEQUENCE_LENGTH, test_size=TEST_SIZE)
    
    model = create_model(trial, input_shape=X_train.shape[1:])
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=MAX_EPOCHS,
        batch_size=trial.suggest_int('batch_size', 32, 256, step=32),
        callbacks=[es],
        verbose=0
    )
    return min(history.history['val_loss'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

SEQUENCE_LENGTH = 30
TEST_SIZE       = 0.2
N_TRIALS        = 50
MAX_EPOCHS      = 50

# 1. Run Optuna hyperparameter search
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS)

print("Best validation MSE:", study.best_value)
for k, v in study.best_trial.params.items():
    print(f"  {k}: {v}")

# 2. Retrain best model on full train+val
(X_train, X_val, y_train, y_val), scaler = prep_data(
    df,
    sequence_length=SEQUENCE_LENGTH,
    test_size=TEST_SIZE
)
best = study.best_trial
best_model = create_model(best, input_shape=X_train.shape[1:])
best_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=MAX_EPOCHS,
    batch_size=best.params['batch_size'],
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)

# 3. Plot predictions vs actual Close (scaled)
preds = best_model.predict(X_val).flatten()
plt.figure(figsize=(10,5))
plt.plot(y_val, label='Actual Close (scaled)')
plt.plot(preds, label='Predicted Close (scaled)')
plt.legend()
plt.title("LSTM Forecast of Next-Day Close Price (Scaled)")
plt.show()

[I 2025-05-04 17:04:30,220] Trial 9 finished with value: 0.017356885597109795 and parameters: {'n_layers': 3, 'n_units_l0': 64, 'dropout_l0': 0.30000000000000004, 'n_units_l1': 160, 'dropout_l1': 0.2, 'n_units_l2': 160, 'dropout_l2': 0.2, 'learning_rate': 1.7473522993777422e-05, 'optimizer': 'Adam', 'batch_size': 128}. Best is trial 1 with value: 0.010943662375211716.
c:\Users\27904\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2025-05-04 17:05:05,661] Trial 10 finished with value: 0.0046534230932593346 and parameters: {'n_layers': 3, 'n_units_l0': 160, 'dropout_l0': 0.0, 'n_units_l1': 32, 'dropout_l1': 0.5, 'n_units_l2': 32, 'dropout_l2': 0.0, 'learning_rate': 0.00010539986058907221, 'optimizer': 'Adam', 'batch_size': 32}. Best 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

SEQUENCE_LENGTH = 30
TEST_SIZE       = 0.2
N_TRIALS        = 50
MAX_EPOCHS      = 50

# 1. Run Optuna hyperparameter search
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS)

print("Best validation MSE:", study.best_value)
for k, v in study.best_trial.params.items():
    print(f"  {k}: {v}")

# 2. Retrain best model on full train+val
(X_train, X_val, y_train, y_val), scaler = prep_data(
    df,
    sequence_length=SEQUENCE_LENGTH,
    test_size=TEST_SIZE
)
best = study.best_trial
best_model = create_model(best, input_shape=X_train.shape[1:])
best_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=MAX_EPOCHS,
    batch_size=best.params['batch_size'],
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)

# 3. Plot predictions vs actual Close (scaled)
preds = best_model.predict(X_val).flatten()
plt.figure(figsize=(10,5))
plt.plot(y_val, label='Actual Close (scaled)')
plt.plot(preds, label='Predicted Close (scaled)')
plt.legend()
plt.title("LSTM Forecast of Next-Day Close Price (Scaled)")
plt.show()

[I 2025-05-04 17:04:30,220] Trial 9 finished with value: 0.017356885597109795 and parameters: {'n_layers': 3, 'n_units_l0': 64, 'dropout_l0': 0.30000000000000004, 'n_units_l1': 160, 'dropout_l1': 0.2, 'n_units_l2': 160, 'dropout_l2': 0.2, 'learning_rate': 1.7473522993777422e-05, 'optimizer': 'Adam', 'batch_size': 128}. Best is trial 1 with value: 0.010943662375211716.
c:\Users\27904\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2025-05-04 17:05:05,661] Trial 10 finished with value: 0.0046534230932593346 and parameters: {'n_layers': 3, 'n_units_l0': 160, 'dropout_l0': 0.0, 'n_units_l1': 32, 'dropout_l1': 0.5, 'n_units_l2': 32, 'dropout_l2': 0.0, 'learning_rate': 0.00010539986058907221, 'optimizer': 'Adam', 'batch_size': 32}. Best 